In [32]:
from pynq import Overlay
from pynq.lib import DMA
from pynq import MMIO
from pynq import Xlnk
import numpy as np


ol = Overlay("/home/xilinx/rosetta/design_1.bit")
ol.download()

dma = ol.axi_dma_0

In [33]:
# USAGE
# python motion_detector.py
# python motion_detector.py --video videos/example_01.mp4

# import the necessary packages
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2

# construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-v", "--video", default="~/rosetta/example_02.mp4",help="path to the video file")
#ap.add_argument("-a", "--min-area", type=int, default=500, help="minimum area size")
#args = vars(ap.parse_args())

# if the video argument is None, then we are reading from webcam
args={"video": "/home/xilinx/rosetta/example_02.mp4", "min-area":500}

if args.get("video", None) is None:
    vs = VideoStream(src=0).start()
    time.sleep(2.0)

# otherwise, we are reading from a video file
else:
    vs = cv2.VideoCapture(args["video"])
    #print(vs)
#vs = cv2.VideoCapture(args["video"])
# initialize the first frame in the video stream
firstFrame = None
#frameBuffer = Xlnk().cma_array(shape=(720, 1152, 3), dtype=np.uint8)
frameBuffer = Xlnk().cma_array(shape=(240, 320, 3), dtype=np.uint32)

# loop over the frames of the video
while True:
    # grab the current frame and initialize the occupied/unoccupied
    # text
    #print(frameBuffer)
    #print(vs.read(frameBuffer))
    frame =vs.read()[1]
    
    #print(frameBuffer)
    #exit()
    #help(vs.read)
    #exit()
    #help(vs.read()[1])
    #print(type(frame))
    #frameBuffer.pointer =  if args.get("video", None) is None else frame[1]
    #text = "Unoccupied"
    # if the frame could not be grabbed, then we have reached the end
    # of the vide
    #print("Start of iteration of while")
    #print(frame)
    
    if frame is None:
        print("Within break block")
        break

    # resize the frame, convert it to grayscale, and blur it
    
    ####################
    
    frame = cv2.resize(frame, (320,240))
    np.copyto(frameBuffer, frame)
    dma.sendchannel.transfer(frameBuffer)
    dma.recvchannel.transfer(frameBuffer)
    #dma.sendchannel.wait()
    #dma.recvchannel.wait()
    #print("After wait")
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #gray = cv2.GaussianBlur(gray, (21, 21), 0)
    ########################
    
    
    # if the first frame is None, initialize it
    if firstFrame is None:
        firstFrame = frameBuffer
        #continue
        pass
    """
    # compute the absolute difference between the current frame and
    # first frame
    frameDelta = cv2.absdiff(firstFrame, frameBuffer)
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]

    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
   
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]

    # loop over the contours
    
    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < args["min-area"]:
            #continue
            pass
    
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frameBuffer, (x, y), (x + w, y + h), (0, 255, 0), 2)
        #text = "Occupied"
    """
    # draw the text and timestamp on the frame
    #cv2.putText(frame, "Room Status: {}".format(text), (10, 20),
    #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    #cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
    #            (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

    # show the frame and record if the user presses a key
    #cv2.imshow("Security Feed", frame)
    #cv2.imshow("Thresh", thresh)
    #cv2.imshow("Frame Delta", frameDelta)
   
    
    #key = cv2.waitKey(1) & 0xFF
    print(frameBuffer.dtype)
    cv2.imwrite("opencv_image.png", frameBuffer.astype(np.uint8))
    break
    # if the `q` key is pressed, break from the lop
    if key == ord("q"):
        break
    
    
# cleanup the camera and close any open windows
vs.stop() if args.get("video", None) is None else vs.release()
#cv2.destroyAllWindows()

uint32


In [4]:
import cv2
cv2